# Part 2A1 Data Preparation & Session Analysis

In [ ]:
# Install required packages
!pip install polars==0.20.31

import polars as pl
import pandas as pd
import numpy as np
import gc
import os
import pickle
import json
import time
import psutil
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Configuration
class Config:
    DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/CML/Assignment 1/content/otto-data'
    OUTPUT_PATH = '/content/drive/MyDrive/Colab Notebooks/CML/Assignment 1/content/otto-output'

    # Memory and performance settings
    TARGET_MEMORY_USAGE_PCT = 70  # Target memory usage percentage
    MIN_CHUNK_SIZE = 1000         # Minimum sessions per chunk
    MAX_CHUNK_SIZE = 50000        # Maximum sessions per chunk

config = Config()

## LOGGING SETUP

In [ ]:
def setup_logging():
    """Setup logging for this notebook"""
    log_file = f"{config.OUTPUT_PATH}/data_preparation_log.txt"

    def log_message(message):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] {message}"
        print(log_entry)

        # Also write to file
        with open(log_file, "a") as f:
            f.write(log_entry + "\n")

    return log_message

log = setup_logging()

log("="*80)
log("OTTO PART 2A1: DATA PREPARATION & SESSION ANALYSIS STARTED")
log("="*80)

[2025-08-07 14:52:59] ================================================================================
[2025-08-07 14:52:59] OTTO PART 2A1: DATA PREPARATION & SESSION ANALYSIS STARTED
[2025-08-07 14:52:59] ================================================================================


## SYSTEM RESOURCE ANALYSIS

In [ ]:
def analyze_system_resources():
    """
    Analyze available system resources to optimize processing

    Returns:
        dict: System resource information
    """
    log("Analyzing system resources...")

    # Memory analysis
    memory = psutil.virtual_memory()
    total_memory_gb = memory.total / (1024**3)
    available_memory_gb = memory.available / (1024**3)
    memory_usage_pct = memory.percent

    # CPU analysis
    cpu_count = psutil.cpu_count()
    cpu_usage = psutil.cpu_percent(interval=1)

    # Disk analysis
    disk = psutil.disk_usage('/')
    disk_free_gb = disk.free / (1024**3)

    log(f"   Memory: {total_memory_gb:.1f} GB total, {available_memory_gb:.1f} GB available ({memory_usage_pct:.1f}% used)")
    log(f"   CPU: {cpu_count} cores, {cpu_usage:.1f}% usage")
    log(f"   Disk: {disk_free_gb:.1f} GB free")

    # Determine optimal chunk size based on available memory
    # Conservative estimation: assume each session needs ~1KB for processing
    estimated_sessions_per_gb = 1000000  # 1M sessions per GB (conservative)
    target_memory_gb = available_memory_gb * (config.TARGET_MEMORY_USAGE_PCT / 100)
    optimal_chunk_size = int(target_memory_gb * estimated_sessions_per_gb)

    # Clamp to reasonable bounds
    optimal_chunk_size = max(config.MIN_CHUNK_SIZE, min(optimal_chunk_size, config.MAX_CHUNK_SIZE))

    log(f"   Optimal chunk size: {optimal_chunk_size:,} sessions")

    resource_info = {
        "timestamp": datetime.now().isoformat(),
        "total_memory_gb": total_memory_gb,
        "available_memory_gb": available_memory_gb,
        "memory_usage_pct": memory_usage_pct,
        "cpu_count": cpu_count,
        "cpu_usage": cpu_usage,
        "disk_free_gb": disk_free_gb,
        "optimal_chunk_size": optimal_chunk_size,
        "target_memory_usage_pct": config.TARGET_MEMORY_USAGE_PCT
    }

    log("System resource analysis completed!")
    return resource_info

## INPUT VALIDATION AND DATA LOADING

In [ ]:
def validate_and_load_training_data():
    """
    Validate input files and load training data

    Returns:
        tuple: (train_features, validation_results)
    """
    log("Validating input files...")

    # Required input files
    required_files = {
        "train_features.parquet": "Processed training data from Part 1"
    }

    # Check if files exist
    missing_files = []
    for filename, description in required_files.items():
        filepath = f"{config.OUTPUT_PATH}/{filename}"
        if not os.path.exists(filepath):
            missing_files.append(f" {filename} - {description}")
        else:
            file_size = os.path.getsize(filepath) / (1024*1024)  # MB
            log(f" {filename} - {file_size:.1f} MB")

    if missing_files:
        log(" MISSING REQUIRED INPUT FILES:")
        for missing in missing_files:
            log(f"   {missing}")
        log("\n TO FIX THIS:")
        log("   Run Part 1 (Data Processing & Feature Engineering) to generate train_features.parquet")
        raise FileNotFoundError("Required input files are missing!")

    log("All required input files found!")

    # Load training data
    log("\n Loading training data...")

    try:
        log("   Loading train_features.parquet...")
        train_features = pl.read_parquet(f"{config.OUTPUT_PATH}/train_features.parquet")
        log(f"    Training data loaded: {train_features.shape} ({train_features.estimated_size('mb'):.1f} MB)")

        # Basic data validation
        log("   Validating data structure...")

        required_columns = ["session", "aid", "ts", "type"]
        missing_columns = [col for col in required_columns if col not in train_features.columns]

        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")

        # Check data types
        log(f"    Data types: {train_features.dtypes}")

        # Basic statistics
        total_events = len(train_features)
        unique_sessions = train_features.select("session").n_unique()
        unique_items = train_features.select("aid").n_unique()

        log(f"    Dataset overview:")
        log(f"      Total events: {total_events:,}")
        log(f"      Unique sessions: {unique_sessions:,}")
        log(f"      Unique items: {unique_items:,}")
        log(f"      Avg events per session: {total_events / unique_sessions:.1f}")

        # Timestamp range
        ts_stats = train_features.select([
            pl.col("ts").min().alias("min_ts"),
            pl.col("ts").max().alias("max_ts")
        ])

        min_ts, max_ts = ts_stats.to_numpy()[0]
        timespan_days = (max_ts - min_ts) / (1000 * 60 * 60 * 24)  # Convert ms to days

        log(f"      Timespan: {timespan_days:.1f} days")

        # Event type distribution
        type_dist = train_features.group_by("type").agg([
            pl.count().alias("count")
        ]).sort("count", descending=True)

        log(f"    Event type distribution:")
        for row in type_dist.iter_rows():
            event_type, count = row
            percentage = count / total_events * 100
            log(f"      {event_type}: {count:,} ({percentage:.1f}%)")

        # Create validation results
        validation_results = {
            "timestamp": datetime.now().isoformat(),
            "total_events": total_events,
            "unique_sessions": unique_sessions,
            "unique_items": unique_items,
            "timespan_days": timespan_days,
            "min_timestamp": int(min_ts),
            "max_timestamp": int(max_ts),
            "avg_events_per_session": total_events / unique_sessions,
            "event_type_distribution": {row[0]: int(row[1]) for row in type_dist.iter_rows()}
        }

        log("Training data validation completed!")
        return train_features, validation_results

    except Exception as e:
        log(f"Error loading training data: {e}")
        raise e

## COMPREHENSIVE SESSION ANALYSIS

In [ ]:
def analyze_session_patterns(train_df: pl.DataFrame) -> Dict:
    """
    Comprehensive analysis of session patterns for co-visitation optimization

    Args:
        train_df: Training data

    Returns:
        dict: Session analysis results
    """
    log("Analyzing session patterns...")

    start_time = time.time()

    # Basic session statistics
    log("   Computing basic session statistics...")

    session_stats = train_df.group_by("session").agg([
        pl.col("aid").count().alias("session_length"),
        pl.col("aid").n_unique().alias("unique_items"),
        pl.col("type").n_unique().alias("unique_types"),
        pl.col("ts").min().alias("session_start"),
        pl.col("ts").max().alias("session_end"),
        pl.col("type").filter(pl.col("type") == "clicks").count().alias("clicks"),
        pl.col("type").filter(pl.col("type") == "carts").count().alias("carts"),
        pl.col("type").filter(pl.col("type") == "orders").count().alias("orders")
    ]).with_columns([
        ((pl.col("session_end") - pl.col("session_start")) / 1000).alias("session_duration_sec")
    ])

    # Session length distribution
    length_stats = session_stats.select([
        pl.col("session_length").min().alias("min_length"),
        pl.col("session_length").max().alias("max_length"),
        pl.col("session_length").mean().alias("avg_length"),
        pl.col("session_length").median().alias("median_length"),
        pl.col("session_length").std().alias("std_length")
    ])

    log(f"    Session length statistics:")
    for row in length_stats.iter_rows():
        min_len, max_len, avg_len, med_len, std_len = row
        log(f"      Min: {min_len}, Max: {max_len}, Avg: {avg_len:.1f}, Median: {med_len:.1f}, Std: {std_len:.1f}")

    # Session duration analysis
    duration_stats = session_stats.select([
        pl.col("session_duration_sec").min().alias("min_duration"),
        pl.col("session_duration_sec").max().alias("max_duration"),
        pl.col("session_duration_sec").mean().alias("avg_duration"),
        pl.col("session_duration_sec").median().alias("median_duration")
    ])

    log(f"     Session duration statistics (seconds):")
    for row in duration_stats.iter_rows():
        min_dur, max_dur, avg_dur, med_dur = row
        log(f"      Min: {min_dur:.0f}, Max: {max_dur:.0f}, Avg: {avg_dur:.0f}, Median: {med_dur:.0f}")

    # Event type patterns within sessions
    log("    Analyzing event type patterns...")

    session_type_patterns = session_stats.with_columns([
        (pl.col("clicks") > 0).alias("has_clicks"),
        (pl.col("carts") > 0).alias("has_carts"),
        (pl.col("orders") > 0).alias("has_orders")
    ])

    pattern_counts = session_type_patterns.select([
        pl.col("has_clicks").sum().alias("sessions_with_clicks"),
        pl.col("has_carts").sum().alias("sessions_with_carts"),
        pl.col("has_orders").sum().alias("sessions_with_orders"),
        (pl.col("has_clicks") & pl.col("has_carts")).sum().alias("click_and_cart"),
        (pl.col("has_clicks") & pl.col("has_orders")).sum().alias("click_and_order"),
        (pl.col("has_carts") & pl.col("has_orders")).sum().alias("cart_and_order"),
        (pl.col("has_clicks") & pl.col("has_carts") & pl.col("has_orders")).sum().alias("all_three_types"),
        pl.count().alias("total_sessions")
    ])

    log(f"    Session type patterns:")
    for row in pattern_counts.iter_rows():
        clicks, carts, orders, click_cart, click_order, cart_order, all_three, total = row
        log(f"      Sessions with clicks: {clicks:,} ({clicks/total*100:.1f}%)")
        log(f"      Sessions with carts: {carts:,} ({carts/total*100:.1f}%)")
        log(f"      Sessions with orders: {orders:,} ({orders/total*100:.1f}%)")
        log(f"      Click + Cart: {click_cart:,} ({click_cart/total*100:.1f}%)")
        log(f"      Click + Order: {click_order:,} ({click_order/total*100:.1f}%)")
        log(f"      Cart + Order: {cart_order:,} ({cart_order/total*100:.1f}%)")
        log(f"      All three types: {all_three:,} ({all_three/total*100:.1f}%)")

    # Session length distribution buckets (FIXED VERSION)
    log("    Session length distribution buckets...")

    # Create length buckets using a different approach to avoid the column error
    length_bucket_data = []
    for row in session_stats.select("session_length").iter_rows():
        session_length = row[0]
        if session_length == 1:
            bucket = "1_item"
        elif session_length <= 5:
            bucket = "2-5_items"
        elif session_length <= 10:
            bucket = "6-10_items"
        elif session_length <= 20:
            bucket = "11-20_items"
        elif session_length <= 50:
            bucket = "21-50_items"
        else:
            bucket = "50+_items"
        length_bucket_data.append(bucket)

    # Create DataFrame with buckets
    bucket_series = pl.Series("length_bucket", length_bucket_data)
    bucket_counts = bucket_series.value_counts().sort("count", descending=True)

    log(f"    Session length distribution:")
    total_sessions = len(length_bucket_data)
    for row in bucket_counts.iter_rows():
        bucket, count = row
        percentage = count / total_sessions * 100
        log(f"      {bucket}: {count:,} sessions ({percentage:.1f}%)")

    # Potential co-visitation opportunities
    log("    Analyzing co-visitation opportunities...")

    # Sessions with multiple items (potential for co-visitation)
    multi_item_sessions = session_stats.filter(pl.col("session_length") > 1).height
    multi_item_pct = multi_item_sessions / total_sessions * 100

    # Sessions with multiple unique items
    multi_unique_sessions = session_stats.filter(pl.col("unique_items") > 1).height
    multi_unique_pct = multi_unique_sessions / total_sessions * 100

    log(f"      Sessions with >1 event: {multi_item_sessions:,} ({multi_item_pct:.1f}%)")
    log(f"      Sessions with >1 unique item: {multi_unique_sessions:,} ({multi_unique_pct:.1f}%)")

    analysis_time = time.time() - start_time

    # Compile analysis results
    analysis_results = {
        "analysis_timestamp": datetime.now().isoformat(),
        "analysis_time_seconds": analysis_time,
        "total_sessions": int(total_sessions),
        "session_length_stats": {
            "min": int(length_stats.select("min_length").item()),
            "max": int(length_stats.select("max_length").item()),
            "mean": float(length_stats.select("avg_length").item()),
            "median": float(length_stats.select("median_length").item()),
            "std": float(length_stats.select("std_length").item())
        },
        "session_duration_stats": {
            "min_seconds": float(duration_stats.select("min_duration").item()),
            "max_seconds": float(duration_stats.select("max_duration").item()),
            "mean_seconds": float(duration_stats.select("avg_duration").item()),
            "median_seconds": float(duration_stats.select("median_duration").item())
        },
        "event_type_patterns": {
            "sessions_with_clicks": int(pattern_counts.select("sessions_with_clicks").item()),
            "sessions_with_carts": int(pattern_counts.select("sessions_with_carts").item()),
            "sessions_with_orders": int(pattern_counts.select("sessions_with_orders").item()),
            "click_and_cart": int(pattern_counts.select("click_and_cart").item()),
            "click_and_order": int(pattern_counts.select("click_and_order").item()),
            "cart_and_order": int(pattern_counts.select("cart_and_order").item()),
            "all_three_types": int(pattern_counts.select("all_three_types").item())
        },
        "length_distribution": {row[0]: int(row[1]) for row in bucket_counts.iter_rows()},
        "covisitation_opportunities": {
            "multi_item_sessions": multi_item_sessions,
            "multi_item_percentage": multi_item_pct,
            "multi_unique_sessions": multi_unique_sessions,
            "multi_unique_percentage": multi_unique_pct
        }
    }

    log(f" Session analysis completed in {analysis_time:.1f} seconds!")
    return analysis_results

## TEMPORAL PATTERN ANALYSIS

In [ ]:
def analyze_temporal_patterns(train_df: pl.DataFrame) -> Dict:
    """
    Analyze temporal patterns for co-visitation timing optimization

    Args:
        train_df: Training data

    Returns:
        dict: Temporal analysis results
    """
    log("Analyzing temporal patterns...")

    start_time = time.time()

    # Sample data for temporal analysis (to save memory)
    sample_size = min(1000000, len(train_df))
    if len(train_df) > sample_size:
        log(f"   Sampling {sample_size:,} events for temporal analysis...")
        sample_df = train_df.sample(sample_size, seed=42)
    else:
        sample_df = train_df

    # Convert timestamps to datetime for analysis
    log("   Converting timestamps for temporal analysis...")

    try:
        sample_pd = sample_df.select(["session", "aid", "ts", "type"]).to_pandas()
        sample_pd['datetime'] = pd.to_datetime(sample_pd['ts'], unit='ms', errors='coerce')
        sample_pd = sample_pd.dropna(subset=['datetime'])

        if len(sample_pd) == 0:
            log("   No valid timestamps for temporal analysis")
            return {"error": "No valid timestamps"}

        # Extract temporal features
        sample_pd['hour'] = sample_pd['datetime'].dt.hour
        sample_pd['day_of_week'] = sample_pd['datetime'].dt.dayofweek
        sample_pd['date'] = sample_pd['datetime'].dt.date

        # Hourly patterns
        log("   Analyzing hourly patterns...")
        hourly_dist = sample_pd['hour'].value_counts().sort_index()

        log(f"   Activity by hour (sample):")
        peak_hours = hourly_dist.nlargest(3)
        for hour, count in peak_hours.items():
            log(f"      Hour {hour}: {count:,} events")

        # Daily patterns
        log("   Analyzing daily patterns...")
        daily_dist = sample_pd['day_of_week'].value_counts().sort_index()

        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        log(f"   Activity by day of week (sample):")
        for day_idx, count in daily_dist.items():
            log(f"      {days[day_idx]}: {count:,} events")

        # Time gaps within sessions
        log("   Analyzing time gaps within sessions...")

        session_time_gaps = []
        for session_id in sample_pd['session'].unique()[:1000]:
            session_data = sample_pd[sample_pd['session'] == session_id].sort_values('ts')
            if len(session_data) > 1:
                time_diffs = session_data['ts'].diff().dropna()
                session_time_gaps.extend(time_diffs.tolist())

        # FIXED: Calculate statistics properly
        if session_time_gaps:
            gap_series = pd.Series(session_time_gaps)
            gap_stats = gap_series.describe()
            p95 = gap_series.quantile(0.95)  # Calculate 95th percentile separately

            log(f"   Time gaps within sessions (ms):")
            log(f"      Mean: {gap_stats['mean']:.0f} ms ({gap_stats['mean']/1000:.1f} sec)")
            log(f"      Median: {gap_stats['50%']:.0f} ms ({gap_stats['50%']/1000:.1f} sec)")
            log(f"      95th percentile: {p95:.0f} ms ({p95/1000:.1f} sec)")

        analysis_time = time.time() - start_time

        # Compile temporal analysis results
        temporal_results = {
            "analysis_timestamp": datetime.now().isoformat(),
            "analysis_time_seconds": analysis_time,
            "sample_size": len(sample_pd),
            "hourly_distribution": hourly_dist.to_dict(),
            "daily_distribution": daily_dist.to_dict(),
            "peak_hours": peak_hours.to_dict(),
            "time_gap_stats": {
                "mean_ms": float(gap_stats['mean']) if session_time_gaps else 0,
                "median_ms": float(gap_stats['50%']) if session_time_gaps else 0,
                "p95_ms": float(p95) if session_time_gaps else 0  # FIXED: Use p95 variable
            } if session_time_gaps else {}
        }

        log(f"Temporal analysis completed in {analysis_time:.1f} seconds!")
        return temporal_results

    except Exception as e:
        log(f"Error in temporal analysis: {e}")
        return {"error": str(e)}

## CHUNKING STRATEGY OPTIMIZATION

In [ ]:
def optimize_chunking_strategy(train_df: pl.DataFrame,
                              session_analysis: Dict,
                              resource_info: Dict) -> Dict:
    """
    Determine optimal chunking strategy for memory-efficient matrix generation

    Args:
        train_df: Training data
        session_analysis: Session analysis results
        resource_info: System resource information

    Returns:
        dict: Chunking strategy configuration
    """
    log("Optimizing chunking strategy...")

    total_sessions = session_analysis["total_sessions"]
    avg_session_length = session_analysis["session_length_stats"]["mean"]
    available_memory_gb = resource_info["available_memory_gb"]
    base_chunk_size = resource_info["optimal_chunk_size"]

    log(f"   Input parameters:")
    log(f"      Total sessions: {total_sessions:,}")
    log(f"      Avg session length: {avg_session_length:.1f}")
    log(f"      Available memory: {available_memory_gb:.1f} GB")
    log(f"      Base chunk size: {base_chunk_size:,}")

    # Estimate memory requirements per session for different matrix types
    # These are conservative estimates based on typical co-visitation patterns

    memory_per_session = {
        "click_to_click": avg_session_length * avg_session_length * 0.1,  # Conservative estimate
        "click_to_buy": avg_session_length * avg_session_length * 0.05,   # Fewer conversions
        "buy_to_buy": avg_session_length * 0.2                            # Even fewer buy events
    }

    # Calculate optimal chunk sizes for each matrix type
    chunk_sizes = {}

    for matrix_type, mem_per_session in memory_per_session.items():
        # Conservative memory allocation (use 50% of available memory)
        target_memory_bytes = available_memory_gb * 0.5 * (1024**3)

        # Estimate sessions that can fit in target memory
        if mem_per_session > 0:
            estimated_sessions = int(target_memory_bytes / mem_per_session)
            chunk_size = max(config.MIN_CHUNK_SIZE, min(estimated_sessions, config.MAX_CHUNK_SIZE))
        else:
            chunk_size = base_chunk_size

        chunk_sizes[matrix_type] = chunk_size

        log(f"   {matrix_type}: {chunk_size:,} sessions per chunk")

    # Calculate number of chunks needed
    chunks_needed = {}
    processing_time_estimates = {}

    for matrix_type, chunk_size in chunk_sizes.items():
        num_chunks = (total_sessions + chunk_size - 1) // chunk_size  # Ceiling division
        chunks_needed[matrix_type] = num_chunks

        # Estimate processing time (rough estimate: 1-5 seconds per chunk)
        estimated_time_minutes = num_chunks * 3 / 60  # 3 seconds per chunk average
        processing_time_estimates[matrix_type] = estimated_time_minutes

        log(f"   {matrix_type}: {num_chunks} chunks, ~{estimated_time_minutes:.1f} minutes")

    # Overall strategy recommendations
    log("   Strategy recommendations:")

    # Memory pressure assessment
    total_estimated_time = sum(processing_time_estimates.values())
    memory_pressure = "HIGH" if available_memory_gb < 16 else "MEDIUM" if available_memory_gb < 32 else "LOW"

    log(f"      Memory pressure: {memory_pressure}")
    log(f"      Total estimated time: {total_estimated_time:.1f} minutes")

    recommendations = []

    if memory_pressure == "HIGH":
        recommendations.extend([
            "Use smaller chunk sizes to avoid memory issues",
            "Process matrices sequentially, not in parallel",
            "Enable aggressive garbage collection between chunks"
        ])
    elif memory_pressure == "MEDIUM":
        recommendations.extend([
            "Standard chunk sizes should work well",
            "Monitor memory usage during processing",
            "Consider processing click-to-click matrix first (largest)"
        ])
    else:
        recommendations.extend([
            "Can use larger chunk sizes for better efficiency",
            "Parallel processing may be possible",
            "Memory should not be a limiting factor"
        ])

    if total_estimated_time > 60:  # > 1 hour
        recommendations.append("Consider running during off-peak hours due to long processing time")

    for rec in recommendations:
        log(f"        {rec}")

    # Compile chunking strategy
    chunking_strategy = {
        "analysis_timestamp": datetime.now().isoformat(),
        "total_sessions": total_sessions,
        "available_memory_gb": available_memory_gb,
        "memory_pressure": memory_pressure,
        "chunk_sizes": chunk_sizes,
        "chunks_needed": chunks_needed,
        "processing_time_estimates": processing_time_estimates,
        "total_estimated_time_minutes": total_estimated_time,
        "recommendations": recommendations,
        "memory_estimates_per_session": memory_per_session
    }

    log("Chunking strategy optimization completed!")
    return chunking_strategy

## DATA OPTIMIZATION FOR MATRIX GENERATION

In [ ]:
def optimize_data_for_covisitation(train_df: pl.DataFrame) -> pl.DataFrame:
    """
    Optimize data representation for efficient co-visitation matrix generation
    Memory-efficient version that processes data in chunks

    Args:
        train_df: Original training data

    Returns:
        pl.DataFrame: Optimized data
    """
    log("Optimizing data for co-visitation matrix generation...")

    start_time = time.time()
    original_size = train_df.estimated_size('mb')

    log(f"   Original data: {train_df.shape}, {original_size:.1f} MB")

    # Step 1: Basic optimizations that don't require window functions
    log("   Applying basic optimizations...")

    # Select only essential columns and optimize data types
    essential_columns = ["session", "aid", "ts", "type"]
    optimized_df = train_df.select(essential_columns).with_columns([
        pl.col("session").cast(pl.UInt32),
        pl.col("aid").cast(pl.UInt32),
        pl.col("ts").cast(pl.UInt64),
        pl.col("type").cast(pl.Categorical)
    ])

    # Sort data for optimal access patterns
    log("   Sorting data for optimal access patterns...")
    optimized_df = optimized_df.sort(["session", "ts"])

    # Step 2: Add memory-efficient derived columns
    log("   Adding essential derived columns...")

    # Add event type codes (memory efficient)
    optimized_df = optimized_df.with_columns([
        pl.when(pl.col("type") == "clicks").then(0)
        .when(pl.col("type") == "carts").then(1)
        .otherwise(2).cast(pl.UInt8).alias("type_code")
    ])

    # Step 3: Process session-level information in chunks to avoid memory issues
    log("   Processing session information in memory-efficient chunks...")

    # Get unique sessions
    unique_sessions = optimized_df.select("session").unique().sort("session")
    total_sessions = len(unique_sessions)
    chunk_size = 100000  # Process 100K sessions at a time

    log(f"   Processing {total_sessions:,} sessions in chunks of {chunk_size:,}...")

    # Pre-calculate session stats more efficiently
    session_stats = optimized_df.group_by("session").agg([
        pl.count().alias("session_length")
    ])

    # Merge session length back efficiently
    optimized_df = optimized_df.join(session_stats, on="session", how="left")

    # Add simple position within session (more memory efficient than ranking)
    log("   Adding session positions efficiently...")

    # Use a simpler approach for session position
    optimized_df = optimized_df.with_columns([
        pl.int_range(pl.len()).over("session").cast(pl.UInt16).alias("session_position")
    ])

    # Cast session_length to appropriate type
    optimized_df = optimized_df.with_columns([
        pl.col("session_length").cast(pl.UInt16)
    ])

    optimization_time = time.time() - start_time
    optimized_size = optimized_df.estimated_size('mb')
    size_change = optimized_size - original_size

    log(f"   Data optimization completed:")
    log(f"      Time: {optimization_time:.1f} seconds")
    log(f"      Size: {optimized_size:.1f} MB (change: {size_change:+.1f} MB)")
    log(f"      Columns: {list(optimized_df.columns)}")

    # Force garbage collection
    gc.collect()

    return optimized_df

## EVENT TYPE ANALYSIS

In [ ]:
def analyze_event_types(train_df: pl.DataFrame) -> Dict:
    """
    Detailed analysis of event type patterns and transitions

    Args:
        train_df: Training data

    Returns:
        dict: Event type analysis results
    """
    log("Analyzing event type patterns and transitions...")

    # Event type distribution
    type_dist = train_df.group_by("type").agg([
        pl.count().alias("count"),
        pl.col("session").n_unique().alias("unique_sessions")
    ]).sort("count", descending=True)

    total_events = len(train_df)
    total_sessions = train_df.select("session").n_unique()

    log("   Event type distribution:")
    event_stats = {}
    for row in type_dist.iter_rows():
        event_type, count, sessions = row
        event_pct = count / total_events * 100
        session_pct = sessions / total_sessions * 100
        event_stats[event_type] = {
            "count": count,
            "percentage": event_pct,
            "unique_sessions": sessions,
            "session_percentage": session_pct
        }
        log(f"      {event_type}: {count:,} events ({event_pct:.1f}%), {sessions:,} sessions ({session_pct:.1f}%)")

    # Event transitions within sessions (sample analysis)
    log("   Analyzing event transitions...")

    try:
        sample_sessions = train_df.select("session").unique().sample(10000, seed=42)["session"].to_list()
        sample_data = train_df.filter(pl.col("session").is_in(sample_sessions)).sort(["session", "ts"])

        # Calculate transitions
        transitions = {}
        sample_pd = sample_data.to_pandas()

        for session_id in sample_sessions[:1000]:
            session_events = sample_pd[sample_pd['session'] == session_id]['type'].tolist()
            if len(session_events) > 1:
                for i in range(len(session_events) - 1):
                    transition = f"{session_events[i]} -> {session_events[i+1]}"
                    transitions[transition] = transitions.get(transition, 0) + 1

        # Sort transitions by frequency
        sorted_transitions = sorted(transitions.items(), key=lambda x: x[1], reverse=True)

        log("   Most common event transitions:")
        for i, (transition, count) in enumerate(sorted_transitions[:10], 1):
            log(f"      {i}. {transition}: {count:,} times")

        transition_data = dict(sorted_transitions[:20])
    except Exception as e:
        log(f"   Error analyzing transitions: {e}")
        transition_data = {}

    event_analysis = {
        "analysis_timestamp": datetime.now().isoformat(),
        "total_events": total_events,
        "total_sessions": total_sessions,
        "event_type_stats": event_stats,
        "top_transitions": transition_data
    }

    log("Event type analysis completed!")
    return event_analysis

## SAVE OUTPUTS

In [ ]:
def save_preparation_outputs(covisit_data: pl.DataFrame,
                           session_analysis: Dict,
                           temporal_analysis: Dict,
                           chunking_strategy: Dict,
                           event_analysis: Dict,
                           validation_results: Dict,
                           resource_info: Dict):
    """
    Save all outputs from data preparation

    Args:
        covisit_data: Optimized data for co-visitation
        session_analysis: Session analysis results
        temporal_analysis: Temporal analysis results
        chunking_strategy: Chunking strategy configuration
        event_analysis: Event type analysis results
        validation_results: Input validation results
        resource_info: System resource information
    """
    log("Saving data preparation outputs...")

    try:
        # 1. Save optimized data (main output)
        data_path = f"{config.OUTPUT_PATH}/covisit_data_prepared.parquet"
        covisit_data.write_parquet(data_path)
        file_size = os.path.getsize(data_path) / (1024*1024)
        log(f"   covisit_data_prepared.parquet saved ({file_size:.1f} MB)")

        # 2. Save session analysis
        session_path = f"{config.OUTPUT_PATH}/session_analysis.json"
        with open(session_path, "w") as f:
            json.dump(session_analysis, f, indent=2)
        log(f"   session_analysis.json saved")

        # 3. Save chunking strategy
        chunking_path = f"{config.OUTPUT_PATH}/chunking_strategy.json"
        with open(chunking_path, "w") as f:
            json.dump(chunking_strategy, f, indent=2)
        log(f"   chunking_strategy.json saved")

        # 4. Save temporal analysis
        temporal_path = f"{config.OUTPUT_PATH}/temporal_analysis.json"
        with open(temporal_path, "w") as f:
            json.dump(temporal_analysis, f, indent=2)
        log(f"   temporal_analysis.json saved")

        # 5. Save event type analysis
        event_path = f"{config.OUTPUT_PATH}/event_type_analysis.json"
        with open(event_path, "w") as f:
            json.dump(event_analysis, f, indent=2)
        log(f"   event_type_analysis.json saved")

        # 6. Save comprehensive summary
        summary = {
            "notebook": "Part 2A1: Data Preparation & Session Analysis",
            "completion_timestamp": datetime.now().isoformat(),
            "system_resources": resource_info,
            "input_validation": validation_results,
            "session_analysis_summary": {
                "total_sessions": session_analysis["total_sessions"],
                "avg_session_length": session_analysis["session_length_stats"]["mean"],
                "covisitation_opportunities": session_analysis["covisitation_opportunities"]
            },
            "chunking_strategy_summary": {
                "memory_pressure": chunking_strategy["memory_pressure"],
                "estimated_total_time": chunking_strategy["total_estimated_time_minutes"],
                "recommended_chunk_sizes": chunking_strategy["chunk_sizes"]
            },
            "optimization_results": {
                "data_size_mb": file_size,
                "columns_added": ["session_position", "session_length", "type_code"],
                "sorting_applied": "session, ts"
            },
            "next_steps": [
                "Run Part 2A2: Click-to-Click Matrix Generation",
                "Use chunking_strategy.json for optimal memory management",
                "Use covisit_data_prepared.parquet as input for matrix generation"
            ]
        }

        summary_path = f"{config.OUTPUT_PATH}/part_2a1_summary.json"
        with open(summary_path, "w") as f:
            json.dump(summary, f, indent=2)
        log(f"   part_2a1_summary.json saved")

        log("All preparation outputs saved successfully!")

        return {
            "data_path": data_path,
            "session_path": session_path,
            "chunking_path": chunking_path,
            "temporal_path": temporal_path,
            "event_path": event_path,
            "summary_path": summary_path
        }

    except Exception as e:
        log(f"Error saving outputs: {e}")
        raise e

## MAIN EXECUTION

In [ ]:
try:
    # Step 1: Analyze system resources
    log("Step 1/7: Analyzing system resources...")
    resource_info = analyze_system_resources()

    # Step 2: Load and validate training data
    log("Step 2/7: Loading and validating training data...")
    train_features, validation_results = validate_and_load_training_data()

    # Step 3: Perform comprehensive session analysis
    log("Step 3/7: Performing session analysis...")
    session_analysis = analyze_session_patterns(train_features)

    # Clean up memory after session analysis
    gc.collect()
    log("   Memory cleanup after session analysis")

    # Step 4: Analyze temporal patterns (with sampling to save memory)
    log("Step 4/7: Analyzing temporal patterns...")
    temporal_analysis = analyze_temporal_patterns(train_features)

    # Clean up memory after temporal analysis
    gc.collect()
    log("   Memory cleanup after temporal analysis")

    # Step 5: Perform event type analysis
    log("Step 5/7: Analyzing event types...")
    event_type_analysis = analyze_event_types(train_features)

    # Clean up memory after event analysis
    gc.collect()
    log("   Memory cleanup after event type analysis")

    # Step 6: Optimize chunking strategy
    log("Step 6/7: Optimizing chunking strategy...")
    chunking_strategy = optimize_chunking_strategy(train_features, session_analysis, resource_info)

    # Step 7: Optimize data for co-visitation processing (MEMORY CRITICAL STEP)
    log("Step 7/7: Optimizing data for co-visitation processing...")
    log("   This is the memory-critical step - monitoring RAM usage...")

    # Check memory before optimization
    memory = psutil.virtual_memory()
    log(f"   Memory before optimization: {memory.percent:.1f}% used, {memory.available/(1024**3):.1f} GB available")

    # Optimize data with memory-efficient version
    covisit_data_prepared = optimize_data_for_covisitation(train_features)

    # Clear the original training data from memory immediately
    log("   Clearing original training data from memory...")
    del train_features
    gc.collect()

    # Check memory after optimization
    memory = psutil.virtual_memory()
    log(f"   Memory after optimization: {memory.percent:.1f}% used, {memory.available/(1024**3):.1f} GB available")

    # Save all outputs
    log("Saving all outputs...")
    output_paths = save_preparation_outputs(
        covisit_data_prepared,
        session_analysis,
        temporal_analysis,
        chunking_strategy,
        event_type_analysis,
        validation_results,
        resource_info
    )

    ## FINAL SUMMARY
    log("\n" + "="*80)
    log("PART 2A1 COMPLETED: DATA PREPARATION & SESSION ANALYSIS")
    log("="*80)

    log(f"\n KEY INSIGHTS:")
    log(f"Total sessions: {session_analysis['total_sessions']:,}")
    log(f"Avg session length: {session_analysis['session_length_stats']['mean']:.1f} events")
    log(f"Co-visitation opportunities: {session_analysis['covisitation_opportunities']['multi_unique_percentage']:.1f}% sessions")
    log(f"Memory pressure: {chunking_strategy['memory_pressure']}")
    log(f"Estimated total processing time: {chunking_strategy['total_estimated_time_minutes']:.1f} minutes")

    log(f"\n OUTPUT FILES GENERATED:")
    for description, path in output_paths.items():
        filename = os.path.basename(path)
        if path.endswith('.parquet'):
            file_size = os.path.getsize(path) / (1024*1024)
            log(f"   {filename} ({file_size:.1f} MB)")
        else:
            log(f"   {filename}")
    log(f"All files saved to: {config.OUTPUT_PATH}")

    log(f"\n OPTIMAL CHUNKING STRATEGY:")
    for matrix_type, chunk_size in chunking_strategy["chunk_sizes"].items():
        chunks = chunking_strategy["chunks_needed"][matrix_type]
        time_est = chunking_strategy["processing_time_estimates"][matrix_type]
        log(f"   {matrix_type}: {chunk_size:,} sessions/chunk, {chunks} chunks, ~{time_est:.1f} min")

    log(f"\n RECOMMENDATIONS:")
    for rec in chunking_strategy["recommendations"]:
        log(f"   {rec}")

    # Final memory cleanup
    del covisit_data_prepared
    gc.collect()

    final_memory = psutil.virtual_memory()
    log(f"\n Final memory usage: {final_memory.percent:.1f}% used, {final_memory.available/(1024**3):.1f} GB available")
    log(f" Part 2A1 finished successfully!")

except Exception as e:
    log(f" ERROR: {e}")
    import traceback
    log(f"Full traceback: {traceback.format_exc()}")

    # Emergency memory cleanup
    try:
        if 'train_features' in locals():
            del train_features
            log("   Emergency cleanup: train_features deleted")
        if 'covisit_data_prepared' in locals():
            del covisit_data_prepared
            log("   Emergency cleanup: covisit_data_prepared deleted")
        gc.collect()
        log("   Emergency memory cleanup completed")

        # Show final memory status
        final_memory = psutil.virtual_memory()
        log(f"   Memory after cleanup: {final_memory.percent:.1f}% used, {final_memory.available/(1024**3):.1f} GB available")
    except Exception as cleanup_error:
        log(f"   Error during emergency cleanup: {cleanup_error}")

    raise e

[2025-08-07 14:52:59] Step 1/7: Analyzing system resources...
[2025-08-07 14:52:59] Analyzing system resources...
[2025-08-07 14:53:00]    Memory: 51.0 GB total, 49.5 GB available (3.0% used)
[2025-08-07 14:53:00]    CPU: 8 cores, 21.7% usage
[2025-08-07 14:53:00]    Disk: 183.2 GB free
[2025-08-07 14:53:00]    Optimal chunk size: 50,000 sessions
[2025-08-07 14:53:00] System resource analysis completed!
[2025-08-07 14:53:00] Step 2/7: Loading and validating training data...
[2025-08-07 14:53:00] Validating input files...
[2025-08-07 14:53:00]  train_features.parquet - 3893.1 MB
[2025-08-07 14:53:00] All required input files found!
[2025-08-07 14:53:00] 
 Loading training data...
[2025-08-07 14:53:00]    Loading train_features.parquet...
[2025-08-07 14:53:33]     Training data loaded: (216384937, 22) (20636.1 MB)
[2025-08-07 14:53:33]    Validating data structure...
[2025-08-07 14:53:33]     Data types: [UInt32, UInt32, UInt64, Categorical(ordering='physical'), UInt32, UInt32, UInt64, U